In [56]:
import requests
import numpy as np
import timeseries as ts
from tsdb import *
from scipy.stats import norm
import json

In [8]:
def tsmaker(m, s, j):
    "returns metadata and a time series in the shape of a jittered normal"
    meta={}
    meta['order'] = int(np.random.choice([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]))
    meta['blarg'] = int(np.random.choice([1, 2]))
    t = np.arange(0.0, 1.0, 0.01)
    v = norm.pdf(t, m, s) + j*np.random.randn(100)
    return meta, ts.TimeSeries(t, v)


In [45]:
meta, ts_ = tsmaker(1, 0.4, 0.2)

In [54]:
def request_insert(pk, ts):
    payload = {'primary_key': pk, 'ts': ts.to_json()}
    r = requests.post("http://127.0.0.1:8080/tsdb/insert_ts", data=json.dumps(payload))
    
def request_upsert_meta(pk, meta):
    payload = {'primary_key': pk, 'md':meta}
    r = requests.post("http://127.0.0.1:8080/tsdb/upsert_meta", data=json.dumps(payload))

def request_select(md={}, fields=None, additional=None):
    payload = {'md':md, 'fields': fields, 'additional': additional}
    r = requests.get("http://127.0.0.1:8080/tsdb/select", data=json.dumps(payload))
    
def request_augmented_select(proc, target, md={}, arg=None, additional=None):
    payload = {'proc':proc, 'target': target, 'md':md, 'arg': arg, 'additional': additional}
    r = requests.get("http://127.0.0.1:8080/tsdb/augmented_select", data=json.dumps(payload))
    
def request_add_trigger(proc, target, onwhat, arg=None):
    payload = {'proc':proc, 'target': target, 'arg': arg}
    r = requests.post("http://127.0.0.1:8080/tsdb/add_trigger", data=json.dumps(payload))
    
def request_remove_trigger(proc, onwhat):
    payload = {'proc':proc, 'target': target, 'arg': arg}
    r = requests.post("http://127.0.0.1:8080/tsdb/remove_trigger", data=json.dumps(payload))
    

In [85]:
# With TSBOp
def request_insert(pk, ts):
    msg = TSDBOp_InsertTS(pk, ts).to_json()
    requests.post("http://127.0.0.1:8080/tsdb/insert_ts", data=json.dumps(msg))

def request_upsert_meta(pk, meta):
    msg = TSDBOp_UpsertMeta(pk, meta).to_json()
    r = requests.post("http://127.0.0.1:8080/tsdb/upsert_meta", data=json.dumps(msg))

def request_select(md={}, fields=None, additional=None):
    msg = TSDBOp_Select(md, fields, additional).to_json()
    r = requests.get("http://127.0.0.1:8080/tsdb/select", data=json.dumps(msg))
    return json.loads(r.text, object_pairs_hook=OrderedDict)
    
def request_augmented_select(proc, target, md={}, arg=None, additional=None):
    msg = TSDBOp_AugmentedSelect(proc, target, arg, md, additional).to_json()
    r = requests.get("http://127.0.0.1:8080/tsdb/augmented_select", data=json.dumps(msg))
    return json.loads(r.text, object_pairs_hook=OrderedDict)
    
def request_add_trigger(proc, target, onwhat, arg=None):
    msg = TSDBOp_AddTrigger(proc, onwhat, target, arg).to_json()
    r = requests.post("http://127.0.0.1:8080/tsdb/add_trigger", data=json.dumps(msg))
    
def request_remove_trigger(proc, onwhat):
    msg = TSDBOp_RemoveTrigger(proc, onwhat).to_json()
    r = requests.post("http://127.0.0.1:8080/tsdb/remove_trigger", data=json.dumps(msg))
    

In [80]:
request_insert('pk_1', ts_)
request_upsert_meta('pk_1', meta)

-----------
-----------


In [81]:
r = request_select()

-----------


In [82]:
r.text

'{"pk_1": {}}'

In [83]:
obj = json.loads(r.text, object_pairs_hook=OrderedDict)

In [84]:
obj

OrderedDict([('pk_1', OrderedDict())])